In [ ]:
# The following should reload files if we change them on disk
%load_ext autoreload
%aimport econsim.work, econsim.resources, econsim.makeragent, econsim.designer, econsim.producer, econsim.serviceprovider, econsim.utils, econsim.distmakingmodel, econsim.market
%autoreload 1

from pdb import set_trace

In [ ]:
from econsim.utils import generate_ranges
from econsim.distmakingmodel import DistMakingModel

price_weight, _quality_ratio, range_living_cost, range_threshold, range_resources = generate_ranges(weight_array=False)

In [ ]:
graphic_params = {
    "method": {
        "type": "SliderInt",
        "value": 1,
        "label": "Profit Sharing Method",
        "min": 1,
        "max": 3,
        "step": 1
    },
    "designers": {
        "type": "SliderInt",
        "value": 40,
        "label": "Designers",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "producers": {
        "type": "SliderInt",
        "value": 12,
        "label": "Producers",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "initial_wealth": {
        "type": "SliderInt",
        "value": 30,
        "label": "Initial Wealth",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "living_cost": {
        "type": "SliderFloat",
        "value": min(range_living_cost),
        "label": "Cost of Living",
        "min": min(range_living_cost),
        "max": max(range_living_cost),
        "step": 0.1
    },
    "resources_amount": {
        "type": "SliderInt",
        "value": min(range_resources),
        "label": "Resources",
        "min": min(range_resources),
        "max": max(range_resources),
        "step": 100
        
    },
    "weights": {
        "type": "SliderFloat",
        "value": min(price_weight),
        "label": "Weight of price",
        "min": min(price_weight),
        "max": max(price_weight),
        "step": 0.1
    },
    "quality_ratio": {
        "type": "SliderFloat",
        "value": 0.5,
        "label": "Ratio quality/sustainability",
        "min": 0.1,
        "max": 1,
        "step": 0.1
    },
        "threshold": {
        "type": "SliderFloat",
        "value": min(range_threshold),
        "label": "Threshold to buy",
        "min": min(range_threshold),
        "max": max(range_threshold),
        "step": 0.05
    },
}



In [ ]:
from econsim.designer import Designer
from econsim.producer import Producer
from econsim.utils import agent_type

def agent_portrayal(agent):
    agtype = agent_type(agent.unique_id)
    if agtype == "Designer":
        if agent.hour_fee == 1:
            color = "#66c2a4"
        elif agent.hour_fee == 1.5:
            color = "#2ca25f"
        elif agent.hour_fee == 2:
            color = "#006d2c"
    elif agtype == "Producer":
        if agent.hour_fee == 1:
            color = "#fc8d59"
        elif agent.hour_fee == 1.5:
            color = "#e34a33"
        elif agent.hour_fee == 2:
            color = "#b30000"
    else:
        color = "tab:green"
    
    size = agent.wealth
    return {"size": size, "color": color, "edge_color": "tab:red"}


In [ ]:
import solara
from matplotlib.figure import Figure

def wealth_histogram(model):
    n_bins = 15
    colors = ["#66c2a4", "#2ca25f", "#006d2c", "#fc8d59", "#e34a33", "#b30000"]
    labels = ["Designers - Low", "Designers - Mid", "Designers - High", "Producers - Low", "Producers - Mid", "Producers - High"]
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    dwealth_low = [agent.wealth for agent in model.schedule.agents if type(agent) == Designer and agent.hour_fee == 1]
    dwealth_mid = [agent.wealth for agent in model.schedule.agents if type(agent) == Designer and agent.hour_fee == 1.5]
    dwealth_high = [agent.wealth for agent in model.schedule.agents if type(agent) == Designer and agent.hour_fee == 2]
    
    pwealth_low = [agent.wealth for agent in model.schedule.agents if type(agent) == Producer and agent.hour_fee == 1]
    pwealth_mid = [agent.wealth for agent in model.schedule.agents if type(agent) == Producer and agent.hour_fee == 1.5]
    pwealth_high = [agent.wealth for agent in model.schedule.agents if type(agent) == Producer and agent.hour_fee == 2]
    
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    # ax.hist(wealth_vals, bins=10)
    ax.hist([dwealth_low,dwealth_mid,dwealth_high,pwealth_low,pwealth_mid, pwealth_high], 
            n_bins, density=True, histtype='bar', stacked=True, color=colors, label=labels)
    ax.legend(prop={'size': 10})
    ax.set_title('Wealth per type agent')
    solara.FigureMatplotlib(fig)

def work_histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.worked_hours for agent in model.schedule.agents if hasattr(agent,'worked_hours')]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)


In [ ]:
from mesa.visualization import SolaraViz

page = SolaraViz(
    DistMakingModel,
    graphic_params,
    measures=[wealth_histogram,
              [
                  "Contributors",
                  # "Hours",
                  "Price",
                  "Quality",
                  "Sustainability",
                  "Material Cost"
              ],
              [
                  # "Designs in Progress",
                  "Realized Designs",
                  # "Products in Progress",
                  "On-sale Products",
                  "Sold Products"
              ],
              "Resources",
              "Gini",
              "Max Score"],
    name="Fair Distribution",
    agent_portrayal=agent_portrayal
)
# This is required to render the visualization in the Jupyter notebook
page